In [1]:
from utils import *
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
try:
    conn = psycopg2.connect("dbname='mimic' user='mimic' host='localhost' password='meladymimic315'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


In [3]:
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

db = np.load('res/itemids.npy').tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

In [4]:
v = np.load('res/filtered_input.npy').tolist()
valid_input = v['id']
valid_input_unit = v['unit']

v = np.load('res/filtered_output.npy').tolist()
valid_output = v['id']

v = np.load('res/filtered_chart.npy').tolist()
valid_chart = v['id']
valid_chart_unit = v['unit']

v = np.load('res/filtered_chart_cate.npy').tolist()
valid_chart_cate = v['id']

v = np.load('res/filtered_lab.npy').tolist()
valid_lab = v['id']
valid_lab_unit = v['unit']

v = np.load('res/filtered_lab_num.npy').tolist()
valid_lab_num = v['id']
valid_lab_num_unit = v['unit']

v = np.load('res/filtered_lab_cate.npy').tolist()
valid_lab_cate = v['id']

v = np.load('res/filtered_microbio.npy').tolist()
valid_microbio = v['id']

v = np.load('res/filtered_prescript.npy').tolist()
valid_prescript = v['id']
valid_prescript_unit = v['unit']


In [5]:
print 'len(valid_input) = '+ str(len(valid_input))
print 'len(valid_output) = '+ str(len(valid_output))
print 'len(valid_chart) = '+ str(len(valid_chart))
print 'len(valid_chart_cate) = '+ str(len(valid_chart_cate))
print 'len(valid_lab) = '+ str(len(valid_lab))
print 'len(valid_lab_num) = '+ str(len(valid_lab_num))
print 'len(valid_lab_cate) = '+ str(len(valid_lab_cate))
print 'len(valid_microbio) = '+ str(len(valid_microbio))
print 'len(valid_prescript) = '+ str(len(valid_prescript))

len(valid_input) = 277
len(valid_output) = 68
len(valid_chart) = 660
len(valid_chart_cate) = 1
len(valid_lab) = 372
len(valid_lab_num) = 54
len(valid_lab_cate) = 231
len(valid_microbio) = 9085
len(valid_prescript) = 2371


In [6]:
allids = valid_input+valid_output+valid_chart+valid_chart_cate+valid_lab+valid_lab_num+valid_lab_cate+valid_microbio+valid_prescript
print len(allids)

13119


In [7]:
admission_ids_txt =""
for t in admission_ids:
    admission_ids_txt += str(t)+","
admission_ids_txt=admission_ids_txt[0:-1]

valid_input_txt =""
for t in valid_input:
    valid_input_txt += str(t)+","
valid_input_txt=valid_input_txt[0:-1]

valid_output_txt =""
for t in valid_output:
    valid_output_txt += str(t)+","
valid_output_txt=valid_output_txt[0:-1]

valid_chart_txt =""
for t in valid_chart:
    valid_chart_txt += str(t)+","
valid_chart_txt=valid_chart_txt[0:-1]

valid_chart_cate_txt =""
for t in valid_chart_cate:
    valid_chart_cate_txt += str(t)+","
valid_chart_cate_txt=valid_chart_cate_txt[0:-1]

valid_lab_txt =""
for t in valid_lab:
    valid_lab_txt += str(t)+","
valid_lab_txt=valid_lab_txt[0:-1]

valid_lab_num_txt =""
for t in valid_lab_num:
    valid_lab_num_txt += str(t)+","
valid_lab_num_txt=valid_lab_num_txt[0:-1]

valid_lab_cate_txt =""
for t in valid_lab_cate:
    valid_lab_cate_txt += str(t)+","
valid_lab_cate_txt=valid_lab_cate_txt[0:-1]

valid_microbio_txt ="'"
for t in valid_microbio:
    valid_microbio_txt += str(t)+"','"
valid_microbio_txt=valid_microbio_txt[0:-2]

valid_prescript_txt ="'"
for t in valid_prescript:
    valid_prescript_txt += str(t)+"','"
valid_prescript_txt=valid_prescript_txt[0:-2]

In [8]:
index =0
map_itemid_index = {}
allitem = valid_output + valid_input + valid_chart + valid_chart_cate + valid_lab + valid_lab_num + valid_lab_cate + valid_microbio + valid_prescript
allitem_unit = ['NOCHECK']*len(valid_output) + valid_input_unit + valid_chart_unit +['NOCHECK'] + valid_lab_unit + valid_lab_num_unit+ ['NOCHECK']*len(valid_lab_cate)+ ['NOCHECK']*len(valid_microbio) + valid_prescript_unit
for i in range(len(allitem_unit)):
    allitem_unit[i] = allitem_unit[i].replace(' ','').lower()
assert len(allitem) == len(allitem_unit)
for ai in allitem:
    map_itemid_index[ai] = index
    index+=1

In [9]:
catedict = {}

for i in valid_chart_cate:
    cur = conn.cursor()
    cur.execute('SELECT distinct value FROM mimiciii.chartevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+')')
    distinctval = cur.fetchall()
    mapping = {};
    ct = 1;
    for d in distinctval:
        mapping[d[0]]= ct;
        ct+=1;
    catedict[i] = mapping
    print '.',
    
for i in valid_lab_cate:
    cur = conn.cursor()
    cur.execute('SELECT distinct value FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+')')
    distinctval = cur.fetchall()
    mapping = {};
    ct = 1;
    for d in distinctval:
        mapping[d[0]]= ct;
        ct+=1;
    catedict[i] = mapping
    print '.',
print "DONE"

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . DONE


In [10]:
def processing(aid,f=sys.stdout):
    
    try:
        conn = psycopg2.connect("dbname='mimic' user='mimic' host='localhost' password='meladymimic315'")
    except:
        f.write("I am unable to connect to the database")
    writeline(f, 'connection established')
    
    # admission
    cur = conn.cursor()
    cur.execute('SELECT * FROM mimiciii.admissions WHERE hadm_id='+str(aid))
    admission = cur.fetchone()
    admitdate = admission[3]
    if(admitdate == None):
        return None;
    writeline(f,"aid : "+ str(aid))
    wholedata = []

    #preprocess inputevents
    cur = conn.cursor()
    cur.execute('SELECT starttime,itemid,amount,amountuom FROM mimiciii.inputevents_mv WHERE amount>0 and hadm_id = '+str(aid)+' and itemid in ('+valid_input_txt+')')
    inputevents = cur.fetchall()
    for ie in inputevents : 
        
        #check date
        if(ie[0] == None):
            writeline(f,'ie date 0 : ' + " : "+str(ie))
            continue;
        
        #discard none value
        if(ie[2] == None):
            writeline(f,'INPUT None value :' + str(ie))
            continue;

        #tuple to list
        ie = list(ie)
        
        #check unit
        currentunit = ie[3].replace(' ','').lower()
        if( currentunit == allitem_unit[map_itemid_index[ie[1]]] or currentunit == ''):
            pass;
        else:    
            gr = findUnitGroup(currentunit)
            if(allitem_unit[map_itemid_index[ie[1]]] in gr):
                targetunit = allitem_unit[map_itemid_index[ie[1]]];
                ie[2] = ie[2] / gr[currentunit] * gr[targetunit]
            else:
                writeline(f,'INPUT Unit not match : ' + ie[3].replace(' ','').lower() + ', orig=' + allitem_unit[map_itemid_index[ie[1]]] + " : "+str(ie))
                continue;

        #discard none value
        if(ie[2] == None):
            writeline(f,'INPUT None value :' + str(ie))
            continue;
            
        wholedata.append(['ie',(ie[0]-admitdate).total_seconds(),list(ie)])

    #preprocess outputevents
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value FROM mimiciii.outputevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_output_txt+')')
    outputevents = cur.fetchall()
    for oe in outputevents : 
        #check date
        if(oe[0] == None):
            writeline(f,'oe date 0 : ' + " : "+str(oe))
            continue;
        
        #discard none value
        if(oe[2] == None):
            writeline(f,'OUTPUT None value :' + str(oe))
            continue;
            
        # no need to check unit all is mL
        
        wholedata.append(['oe',(oe[0]-admitdate).total_seconds(),list(oe)])

    #preprocess chartevents
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,valuenum,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_chart_txt+')')
    chartevents = cur.fetchall()
    for ce in chartevents: 
        
        #check date
        if(ce[0] == None):
            writeline(f,'ce date 0 : ' + " : "+str(ce))
            continue;
        
        #discard none value
        if(ce[2] == None):
            writeline(f,'CHART None value :' + str(ce))
            continue;
        
        #tuple to list
        ce = list(ce)
        
        #check unit
        currentunit = ce[3].replace(' ','').lower()
        if( currentunit == allitem_unit[map_itemid_index[ce[1]]] or currentunit == ''):
            pass;
        else:    
            gr = findUnitGroup(currentunit)
            if(allitem_unit[map_itemid_index[ce[1]]] in gr):
                targetunit = allitem_unit[map_itemid_index[ce[1]]];
                ce[2] = ce[2] / gr[currentunit] * gr[targetunit]
                writeline(f,'******************************************' + str(ce))
            else:
                writeline(f,'CHART Unit not match : ' + ce[3].replace(' ','').lower() + ', orig=' + allitem_unit[map_itemid_index[ce[1]]] + ' : ' + " : "+str(ce))
                continue;
        
        #discard none value
        if(ce[2] == None):
            writeline(f,'CHART None value :' + str(ce))
            continue;
        
        wholedata.append(['ce',(ce[0]-admitdate).total_seconds(),list(ce)])

        
    #preprocess chartevents cate
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_chart_cate_txt+')')
    chartevents = cur.fetchall()
    for ce in chartevents: 
        
        #check date
        if(ce[0] == None):
            writeline(f,'cecate date 0 : ' + " : "+str(ce))
            continue;
        
        #discard none value
        if(ce[2] == None):
            writeline(f,'cecate None value :' + str(ce))
            continue;
        
        #tuple to list
        ce = list(ce)
        
        #map to num
        ce[2] = catedict[ce[1]][ce[2]]
        wholedata.append(['cecate',(ce[0]-admitdate).total_seconds(),list(ce)])

            
    #preprocess labevents
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,valuenum,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_lab_txt+')')
    labevents = cur.fetchall()
    for le in labevents :    
        #check date
        if(le[0] == None):
            writeline(f,'le date 0 : ' + " : "+str(le))
            continue;

        #discard none value
        if(le[2] == None):
            writeline(f,'LAB None value :' + str(le))
            continue;
    
        #tuple to list
        le = list(le)
        
        #check unit
        currentunit = le[3].replace(' ','').replace('<','').replace('>','').replace('=','').lower()
        if( currentunit == allitem_unit[map_itemid_index[le[1]]] or currentunit == ''):
            pass;
        else:    
            gr = findUnitGroup(currentunit)
            if(allitem_unit[map_itemid_index[le[1]]] in gr):
                targetunit = allitem_unit[map_itemid_index[le[1]]];
                le[2] = le[2] / gr[currentunit] * gr[targetunit]
            else:
                writeline(f,'LAB Unit not match : ' + le[3].replace(' ','').lower() + ', orig=' + allitem_unit[map_itemid_index[le[1]]] + " : " + " : "+str(le))
                continue;
        
        #discard none value
        if(le[2] == None):
            writeline(f,'LAB None value :' + str(le))
            continue;
        
        #discard none value
        if(le[2] == None):continue;
        
        wholedata.append(['le',(le[0]-admitdate).total_seconds(),list(le)])
        
    #preprocess labevents_num
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_lab_num_txt+')')
    labevents = cur.fetchall()
    for le in labevents :    
        
        #tuple to list
        le = list(le)
        
        #translate values
        le[2] = parseNum(le[2]);
        
        #check date
        if(le[0] == None):
            writeline(f,'lenum date 0 : ' + " : "+str(le))
            continue;

        #discard none value
        if(le[2] == None):
            writeline(f,'lenum None value :' + str(le))
            continue;
    
        #check unit
        currentunit = le[3].replace(' ','').replace('<','').replace('>','').replace('=','').lower()
        if( currentunit == allitem_unit[map_itemid_index[le[1]]] or currentunit == ''):
            pass;
        else:    
            gr = findUnitGroup(currentunit)
            if(allitem_unit[map_itemid_index[le[1]]] in gr):
                targetunit = allitem_unit[map_itemid_index[le[1]]];
                le[2] = le[2] / gr[currentunit] * gr[targetunit]
            else:
                writeline(f,'lenum Unit not match : ' + le[3].replace(' ','').lower() + ', orig=' + allitem_unit[map_itemid_index[le[1]]] + " : " + " : "+str(le))
                continue;
        
        #discard none value
        if(le[2] == None):
            writeline(f,'lenum None value :' + str(le))
            continue;
        
        #discard none value
        if(le[2] == None):continue;
        
        wholedata.append(['lenum',(le[0]-admitdate).total_seconds(),list(le)])

    #preprocess chartevents num
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_lab_cate_txt+')')
    labevents = cur.fetchall()
    for le in labevents :    
        
        #tuple to list
        le = list(le)
        
        #check date
        if(le[0] == None):
            writeline(f,'lecate date 0 : ' + " : "+str(le))
            continue;

        #discard none value
        if(le[2] == None):
            writeline(f,'lecate None value :' + str(le))
            continue;
        
        le[2] = catedict[le[1]][le[2]];
        
        wholedata.append(['lecate',(le[0]-admitdate).total_seconds(),list(le)])
        
        
    #preprocess prescriptions
    cur = conn.cursor()
    cur.execute('SELECT starttime,formulary_drug_cd,dose_val_rx,dose_unit_rx FROM mimiciii.prescriptions WHERE hadm_id = '+str(aid)+' and formulary_drug_cd in ('+valid_prescript_txt+')')
    presevents = cur.fetchall()
    for pe in presevents: 
        #checkdate
        if(pe[0] == None):
            writeline(f,'pe date 0 : '  + " : "+str(pe))
            continue;
        
        #tuple to list
        pe = list(pe)
        
        # formatting the value
        dose = pe[2]
        dose = dose.replace(',','').replace(' ','')
        numVal = None;
        try:
            numVal = float(dose)
        except:
            if(len(dose.split('-'))==2):
                strs = dose.split('-')
                try:
                    numVal = (float(strs[0]) + float(strs[1]))/2.0
                except:
                    writeline(f,'pe parse fail : '  + " : "+str(pe))
                    continue;
            else:
                writeline(f,'pe parse fail : '  + " : "+str(pe))
                continue;
        
        pe[2] = numVal;
        
        #discard none value
        if(pe[2] == None):
            writeline(f,'PRES None value :' + str(pe))
            continue;
            
        #check unit
        currentunit = pe[3].replace(' ','').lower()
        if( currentunit == allitem_unit[map_itemid_index[pe[1]]] or currentunit == ''):
            pass;
        else:    
            gr = findUnitGroup(currentunit)
            if(allitem_unit[map_itemid_index[pe[1]]] in gr):
                targetunit = allitem_unit[map_itemid_index[pe[1]]];
                pe[2] = pe[2] / gr[currentunit] * gr[targetunit]
            else:
                writeline(f,'PRES Unit not match : ' + pe[3].replace(' ','').lower() + ', orig=' + allitem_unit[map_itemid_index[pe[1]]]+ " : "+str(pe))
                continue;

        #discard none value
        if(pe[2] == None):
            writeline(f, 'PRES None value :' + str(pe))
            continue;
        
        wholedata.append(['pe',(pe[0]-admitdate).total_seconds(),list(pe)])

    #micro biology preprocess
    for mid in valid_microbio:
        mid = mid.replace('(','').replace(')','')
        m = mid.split(',')
        cur = conn.cursor()
        sql = 'SELECT charttime,(spec_itemid,org_itemid,ab_itemid),dilution_text,\'uom\' FROM mimiciii.microbiologyevents WHERE hadm_id = '+str(aid)
        
        if(m[0] == ''):
            sql += ' and spec_itemid is null'
        else:
            sql += ' and spec_itemid = '+m[0]
    
        if(m[1] == ''):
            sql += ' and org_itemid is null'
        else:
            sql += ' and org_itemid = '+m[1]
        
        if(m[2] == ''):
            sql += ' and ab_itemid is null'
        else:
            sql += ' and ab_itemid = '+m[2]

        cur.execute(sql)
        microevents = cur.fetchall()
        for me in microevents: 
            
            #checkdate
            if(me[0] == None):
                writeline(f,'me date 0 : '+ " : "+str(me))
                continue;

            #discard none value
            if(me[2] == None):
                writeline(f,'MICRO None value :' + str(me))
                continue;
            
            #tuple to list
            me = list(me)
            
            #formatting 
            dose = me[2]
            dose = dose.replace('<','').replace('>','').replace('=','')
            numVal = None;
            if(dose == ''):
                writeline(f,'me parse fail : '+ " : "+str(me))
                continue;
            try:
                numVal = float(dose)
            except:
                writeline(f,'me parse fail : '+ " : "+str(me))
                continue;

            me[2]=numVal;
            
            #discard none value; check again after process
            if(me[2] == None):
                writeline(f,'MICRO None value :' + str(me))
                continue;
                    
            wholedata.append(['me',(me[0]-admitdate).total_seconds(),list(me)])

    # sort by time
    wholedata = sorted(wholedata, key=itemgetter(1))

    # index time
    map_time_index = {}
    index = 0;
    for wd in wholedata:
        if(wd[1] not in map_time_index):
            map_time_index[wd[1]] = index
            index += 1;

    # date input output labevent prescript
    D = 2+len(valid_input)+len(valid_output)+len(valid_chart)+len(valid_chart_cate)+len(valid_lab)+len(valid_lab_num)+len(valid_lab_cate)+len(valid_microbio)+len(valid_prescript)

    patient = [[None for i in range(D)] for j in range(len(map_time_index))]
    numtodivide = [[0 for i in range(D-2)] for j in range(len(map_time_index))]
    writeline(f,'len(wholedata) = '+str(len(wholedata)))
    writeline(f, 'D = '+str(D))
    writeline(f,'len(patient) = '+str(len(patient)) +' timesteps')

    for wd in wholedata:

        assert patient[ map_time_index[wd[1]] ][D-2] == None or patient[ map_time_index[wd[1]] ][D-2] == wd[1]
        patient[ map_time_index[wd[1]] ][D-2] = wd[1]
        patient[ map_time_index[wd[1]] ][D-1] = aid

        if(wd[0] ==  'ie' or wd[0] ==  'oe' or wd[0] ==  'pe'):
            if(patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]]] ==None):
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]]] = wd[2][2] 
            else: 
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]]] += wd[2][2]

        if(wd[0] ==  'le' or wd[0] == 'ce' or wd[0] == 'me' or wd[0] ==  'lenum'):
            if(patient[map_time_index[wd[1]]] [map_itemid_index[wd[2][1]]] == None):
                patient[  map_time_index[wd[1]]  ][  map_itemid_index[wd[2][1]]  ] = wd[2][2]
                numtodivide[map_time_index[wd[1]]  ]  [  map_itemid_index[wd[2][1]] ] = 1
            else: 
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]]] += wd[2][2]
                numtodivide[ map_time_index[wd[1]]  ] [   map_itemid_index[wd[2][1]] ] += 1
        
        if(wd[0] == 'cecate' or wd[0] == 'lecate'):
            if(patient[map_time_index[wd[1]]] [map_itemid_index[wd[2][1]]] == None):
                patient[  map_time_index[wd[1]]  ][  map_itemid_index[wd[2][1]]  ] = wd[2][2]
            else: 
                print 'DUPLICATED :',wd
        
    for i in range(len(map_time_index)):
        for j in range(D-2):
            if(numtodivide[i][j] == 0): continue;
            patient[i][j] /= numtodivide[i][j]
    conn.close()
    return patient


In [ ]:
# testing is cruel, testing is cruel ...
# tons of indexes make me sick !
# once coded, dont touch !

print "parseTest =",parseNum("1")==1
print "parseTest =", parseNum("1.1")==1.1
print "parseTest =", parseNum(".2")==.2
print "parseTest =", parseNum("0.2")==.2
print "parseTest =", parseNum(".21")==.21
print "parseTest =", parseNum("0.21")==.21
print "parseTest =", parseNum("1.0-2.0")==1.5
print "parseTest =", parseNum("1-2")==1.5
print "parseTest =", parseNum("dsavdsav 1-2 dasvdsavsdav")==1.5
print "parseTest =", parseNum("dsavdsav 1.0-2.0 dasvdsavsdav")==1.5
print "parseTest =", parseNum("dsavdsav 1.-2. dasvdsavsdav")==1.5
print "parseTest =", parseNum("dsavdsav 1 - 2 dasvdsavsdav")==1.5
print "parseTest =", parseNum("dsavdsav 1.0 - 2.0 dasvdsavsdav")==1.5
print "parseTest =", parseNum("dsavdsav 1. - 2. dasvdsavsdav")==1.5

#general
# 1. all timestep aligns
aid = 100416
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()

admit_date = adm[3]
birthdate = pat[3]


st = -999999999999999
D = len(res[0])
for i in range(len(res)):
    r = list(res[i])
    if( r[D-2] > st):
        st = r[D-2]
    else:
        print'************************************************************************************************g1 FAIL'
print'************************************************************************************************g1 PASS'

# 2. all id is correct
for i in range(len(res)):
    r = list(res[i])
    if( r[D-1] == aid):
        st = r[D-2]
    else:
        print'************************************************************************************************g2 FAIL'
print'************************************************************************************************g2 PASS'

#inputevents
# 1. place correctly at certain timestep
# testcase rowid = 1553769
cur = conn.cursor()
cur.execute('select * from mimiciii.inputevents_mv where row_id = 1553769')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if( math.fabs(r[map_itemid_index[220950]] - 1211.9) < 0.1): test=True
        break;
if(test):
    print'************************************************************************************************i1 PASS' 
else:
    print'************************************************************************************************i1 FAIL'


# 2. sum up correctly at certain timestep
cur = conn.cursor()
cur.execute('select * from mimiciii.inputevents_mv where row_id = 1553421')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if( math.fabs(r[map_itemid_index[220950]] - 628.) < 0.1): test=True
        break;
if(test):
    print'************************************************************************************************i2 PASS' 
else:
    print'************************************************************************************************i2 FAIL'


# 3. the one that has no date doesnot appear
# NO SUCH CASE
cur = conn.cursor()
cur.execute('select count(*) from mimiciii.inputevents_mv where starttime is null')
tc = cur.fetchone()
if( tc[0] == 0):
    print'************************************************************************************************i3 PASS' 
else:
    print'************************************************************************************************i3 FAIL' 
# 4. value == None does not appear
# NO SUCH CASE
cur = conn.cursor()
cur.execute('select count(*) from mimiciii.inputevents_mv where amount is null')
tc = cur.fetchone()
if( tc[0] == 0):
    print'************************************************************************************************i4 PASS' 
else:
    print'************************************************************************************************i4 FAIL' 

# 5. unit is changed correctly.
aid = 149609
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.inputevents_mv where row_id = 498608')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if( math.fabs( r[map_itemid_index[tc[6]]] - 4.8066) < 0.0001): test=True
        break;
if(test):
    print'************************************************************************************************i5 PASS' 
else:
    print'************************************************************************************************i5 FAIL'



#outputevents
# 1. place correctly at certain timestep
cur = conn.cursor()
cur.execute('select * from mimiciii.outputevents where row_id = 3008938')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if( math.fabs( r[map_itemid_index[tc[5]]] - tc[6]) < 0.0001): test=True
        break;
if(test):
    print'************************************************************************************************o1 PASS' 
else:
    print'************************************************************************************************o1 FAIL'

# 2. sum up correctly at certain timestep
# NO SUCH CASE
cur = conn.cursor()
cur.execute('select hadm_id, itemid,charttime from mimiciii.outputevents where itemid in ('+valid_output_txt+') group by hadm_id, itemid,charttime having count(itemid)>1')
tc = cur.fetchall()
if( len(tc) == 0):
    print'************************************************************************************************o2 PASS' 
else:
    print'************************************************************************************************o2 FAIL' 
    
# 3. the one that has no date doesnot appear
# NOSUCH CASE
cur = conn.cursor()
cur.execute('select count(*) from mimiciii.outputevents where charttime is null')
tc = cur.fetchone()
if( tc[0] == 0):
    print'************************************************************************************************o3 PASS' 
else:
    print'************************************************************************************************o3 FAIL' 

# 4. value == None does not appear
cur = conn.cursor()
cur.execute('select count(*) from mimiciii.outputevents where hadm_id in ('+admission_ids_txt+') and value is null and itemid in ('+valid_output_txt+')' )
tc = cur.fetchone()
if( tc[0] == 0):
    print'************************************************************************************************o4 PASS' 
else:
    print'************************************************************************************************o4 FAIL' 


#chartevents
# 1. place correctly at certain timestep
aid = 145708
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.chartevents where row_id = 31862691')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[5]-admit_date).total_seconds()):
        print  tc[8]
        if( math.fabs( r[map_itemid_index[tc[4]]] - tc[9]) < 0.0001): test=True
        break;
if(test):
    print'************************************************************************************************c1 PASS' 
else:
    print'************************************************************************************************c1 FAIL'

# 2. average correctly at certain timestep
aid = 130940
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.chartevents where row_id = 33326746')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[5]-admit_date).total_seconds()):
        print  tc[8]
        if( math.fabs( r[map_itemid_index[tc[4]]] - tc[9]) < 0.0001): test=True
        break;
if(test):
    print'**************************CANT FIND GOOD TESTCASE***********************************************c2 PASS*' 
else:
    print'************************************************************************************************c2 FAIL'

# 3. the one that has no date doesnot appear
cur = conn.cursor()
cur.execute('select count(*) from mimiciii.chartevents where charttime is null')
tc = cur.fetchone()
if( tc[0] == 0):
    print'************************************************************************************************c3 PASS' 
else:
    print'************************************************************************************************c3 FAIL' 


# 4. value == None does not appear

sql = 'select * from mimiciii.chartevents where valuenum is null and itemid in ('+ valid_chart_txt+') and hadm_id in ('+admission_ids_txt+')'
cur = conn.cursor()
cur.execute(sql)
tc = cur.fetchall()
print tc
if(len(tc) > 1):
    print'************************************************************************************************c4 FAIL' 
else:
    print'************************************************************************************************c4 PASS'
    

# 5. unit is changed correctly.
print'************************************************************************************************c4 CANT FIND TESTCASE'


#labevents
# 1. place correctly at certain timestep

aid = 138868
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.labevents where row_id = 10409120')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        print  tc[6]
        if( math.fabs( r[map_itemid_index[tc[3]]] - tc[6]) < 0.0001): test=True
        break;
if(test):
    print'************************************************************************************************l1 PASS' 
else:
    print'************************************************************************************************l1 FAIL'

# 2. average correctly at certain timestep
print'************************************************************************************************l2 TODO*****' 

# 3. the one that has no date doesnot appear
cur = conn.cursor()
cur.execute('select count(*) from mimiciii.labevents where charttime is null')
tc = cur.fetchone()
if( tc[0] == 0):
    print'************************************************************************************************l3 PASS' 
else:
    print'************************************************************************************************l3 FAIL' 

# 4. value == None does not appear
aid = 156425
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.labevents where row_id = 7011460')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if(r[map_itemid_index[tc[3]]] == None): test=True
        break;
if(test):
    print'************************************************************************************************l4 PASS' 
else:
    print'************************************************************************************************l4 FAIL'

#prescript
# 1. parse correctly with - by doing average
aid = 190444
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.prescriptions where row_id = 4086682')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if( math.fabs( r[map_itemid_index['TRAZ50']] - 37.5) < 0.0001): test=True
        break;

if(test):
    print'************************************************************************************************p1.1 PASS' 
else:
    print'************************************************************************************************p1.1 FAIL'

    
cur = conn.cursor()
cur.execute('select * from mimiciii.prescriptions where row_id = 4086643')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if( math.fabs( r[map_itemid_index['CLON5']] - .5) < 0.0001): test=True
        break;

if(test):
    print'************************************************************************************************p1.2 PASS' 
else:
    print'************************************************************************************************p1.2 FAIL'

# 2. place correctly at certain timestep

# 3. sum correctly at certain timestep

cur = conn.cursor()
cur.execute('select * from mimiciii.prescriptions where row_id = 4086640')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        print  r[map_itemid_index['LABE100I']]
        if( math.fabs( r[map_itemid_index['LABE100I']] - 65) < 0.0001): test=True
        break;

if(test):
    print'************************************************************************************************p3 PASS' 
else:
    print'************************************************************************************************p3 FAIL'

# 4. unit is changed correctly.
# translate unit ml -> mL

aid = 196250
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.prescriptions where row_id = 3683278')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if( math.fabs( r[map_itemid_index['PROP200IG']] - 20./1000) < 0.00001): test=True
        break;
if(test):
    print'************************************************************************************************p4 PASS' 
else:
    print'************************************************************************************************p4 FAIL'

# reject unit  dose + mg

aid = 122781
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.prescriptions where row_id = 751415')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if( r[map_itemid_index['VIAL']] == None): test=True
        break;
if(test):
    print'************************************************************************************************p4 PASS' 
else:
    print'************************************************************************************************p4 FAIL'

#microbiology
# 1. parse correctly remove <= >= = to **float**

aid = 139988
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.microbiologyevents where row_id = 3894')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if( r[map_itemid_index['(70091,80293,90007)']] == 0.25): test=True
        break;
        
if(test):
    print'************************************************************************************************m1 PASS' 
else:
    print'************************************************************************************************m1 FAIL'

# 2. average correctly

aid = 166584
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.microbiologyevents where row_id = 252389')
tc = cur.fetchone()
test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (tc[4]-admit_date).total_seconds()):
        if( r[map_itemid_index['(70062,80002,90012)']] == 12): test=True
        break;
        
if(test):
    print'************************************************************************************************m2 PASS' 
else:
    print'************************************************************************************************m2 FAIL'

    
aid = 132173
res = processing (aid)
cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(aid))
adm = cur.fetchone()
cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(adm[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

cur = conn.cursor()
cur.execute('select * from mimiciii.labevents where row_id = '+str(18416208))
lab = cur.fetchone()
res = processing (lab[2]);

for i in range(len(valid_lab_num)):
    if(valid_lab_num[i] == lab[3]):
        break;

if(valid_lab_num_unit[i] == lab[7]) :
    print'************************************************************************************************labnum1 PASS' 
else:
    print'************************************************************************************************labnum1 FAIL'

test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (lab[4]-admit_date).total_seconds()):
        if( r[map_itemid_index[lab[3]]] == 1): test=True
        break;
        
if(test) :
    print'************************************************************************************************labnum2 PASS' 
else:
    print'************************************************************************************************labnum2 FAIL'


cur = conn.cursor()
cur.execute('select hadm_id,itemid,charttime from mimiciii.labevents where hadm_id is not null and itemid in (51475,50845,51280,50935,51479,50922,51501,50925,50856,50981,51213,50915,51497,51046,50835,51176,51180,51194,51196,51130,51131,51132,50906,51076,51422,51517,51229,50946,51471,50899,51515,51369,50992,50958,50926,50961,51228,50877,51494,50990,50991,51489,51488,51061,51225,50894,50989,51209,51516,51493,51476,50911,51003,51482) group by hadm_id,itemid,charttime having count(*) >1')
avgs = cur.fetchall()
hasCase = False;
for avg in avgs:
    if(int(avg[0]) in admission_ids):
        hasCase = True;

if(hasCase) :
    print'************************************************************************************************labnum3 FAIL' 
else:
    print'************************************************************************************************labnum3 PASS NO TC'


    
    
# check category
cur = conn.cursor()
cur.execute('select * from mimiciii.labevents where row_id = '+str(23206431))
lab = cur.fetchone()
res = processing (lab[2]);

cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(lab[2]))
adm = cur.fetchone()

cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(lab[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (lab[4]-admit_date).total_seconds()):
        if( r[map_itemid_index[lab[3]]] == catedict[lab[3]][lab[5]]): 
            test=True
        break;

if(test) :
    print'************************************************************************************************labcate1 PASS' 
else:
    print'************************************************************************************************labcate1 FAIL'

    
# chart category
cur = conn.cursor()
cur.execute('select * from mimiciii.chartevents where row_id = '+str(257475189))
chart = cur.fetchone()
res = processing (chart[2]);

cur = conn.cursor()
cur.execute('select * from mimiciii.admissions where hadm_id = '+str(chart[2]))
adm = cur.fetchone()

cur = conn.cursor()
cur.execute('select * from mimiciii.patients where subject_id = '+str(chart[1]))
pat = cur.fetchone()
admit_date = adm[3]
birthdate = pat[3]

test = False
for i in range(len(res)):
    r = list(res[i])
    if(r[D-2] == (chart[5]-admit_date).total_seconds()):
        print r[map_itemid_index[chart[4]]] 
        if( r[map_itemid_index[chart[4]]] == catedict[chart[4]][chart[8]]): 
            test=True
        break;

if(test) :
    print'************************************************************************************************chartcate2 PASS' 
else:
    print'************************************************************************************************chartcate2 FAIL'


    

parseTest = True
parseTest = True
parseTest = True
parseTest = True
parseTest = True
parseTest = True
parseTest = True
parseTest = True
parseTest = True
parseTest = True
parseTest = True
parseTest = True
parseTest = True
parseTest = True
connection established
aid : 100416
INPUT Unit not match : mg, orig=dose : [datetime.datetime(2162, 7, 1, 19, 50), 225884, 0.001, 'mg']
LAB None value :(datetime.datetime(2162, 6, 23, 18, 5), 51498, None, ' ')
len(wholedata) = 10784
D = 13121
len(patient) = 1364 timesteps
DUPLICATED : ['cecate', 8820.0, [datetime.datetime(2162, 6, 24, 0, 21), 223758, 2, '']]
DUPLICATED : ['cecate', 8820.0, [datetime.datetime(2162, 6, 24, 0, 21), 223758, 2, '']]
DUPLICATED : ['cecate', 52920.0, [datetime.datetime(2162, 6, 24, 12, 36), 223758, 2, '']]
DUPLICATED : ['cecate', 52920.0, [datetime.datetime(2162, 6, 24, 12, 36), 223758, 2, '']]
DUPLICATED : ['cecate', 98400.0, [datetime.datetime(2162, 6, 25, 1, 14), 223758, 2, '']]
DUPLICATED : ['cecate', 98400.0, [datetime.d